In [2]:
import findspark
findspark.init()
import pyspark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

#create the Spark Session
spark = SparkSession.builder.getOrCreate()
#create the Spark Context
sc = spark.sparkContext

* UBER almacena en un cluster todos los datos sobre el movimiento y viajes de todos sus vehículos. Existe un proceso que nos devuelve un RDD llamado trip_summary con los siguientes campos: (driver_id, car_id, trip_id, customer_id, date (YYYYMMDD), distance_traveled), Programar usando Py Spark un programa que nos indique cual fue el conductor con mayor promedio de distancia recorrida por viaje para Abril de 2016.

In [16]:
trips = [
    (1, 1, 1, 1, '20160101', 10),
    (2, 2, 2, 2, '20160202', 20),
    (1, 1, 3, 1, '20160402', 15),
    (1, 1, 4, 3, '20160405', 20),
    (2, 2, 5, 4, '20160410', 25),
    (3, 3, 6, 3, '20160415', 15),
    (2, 2, 7, 1, '20160420', 40),
    (3, 3, 8, 2, '20160505', 80)
]

In [17]:
data = sc.parallelize(trips);

In [18]:
data.collect() # Ver la data pero no recomendable hacerlo

[(1, 1, 1, 1, '20160101', 10),
 (2, 2, 2, 2, '20160202', 20),
 (1, 1, 3, 1, '20160402', 15),
 (1, 1, 4, 3, '20160405', 20),
 (2, 2, 5, 4, '20160410', 25),
 (3, 3, 6, 3, '20160415', 15),
 (2, 2, 7, 1, '20160420', 40),
 (3, 3, 8, 2, '20160505', 80)]

In [21]:
# Solo pide de abril de 2016
data.filter(lambda x: x[4] > '20160400' and x[4] < '20160500').collect()

[(1, 1, 3, 1, '20160402', 15),
 (1, 1, 4, 3, '20160405', 20),
 (2, 2, 5, 4, '20160410', 25),
 (3, 3, 6, 3, '20160415', 15),
 (2, 2, 7, 1, '20160420', 40)]

In [37]:
# Para agrupar por conductor hacemos que la clave sea el id como (id, (distancia recorrida, 1))
data.filter(lambda x: x[4] > '20160400' and x[4] < '20160500').map(lambda x: (x[0], (x[5], 1))).collect()

[(1, (15, 1)), (1, (20, 1)), (2, (25, 1)), (3, (15, 1)), (2, (40, 1))]

In [40]:
data.filter(lambda x: x[4] > '20160400' and x[4] < '20160500').map(lambda x: (x[0],(x[5],1)))\
    .reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))\
    .reduce(lambda x, y: x if (x[1][0] / x[1][1]) > (y[1][0] / y[1][1]) else y)

(2, (65, 2))

* Una red social almacena el contenido de los chats entre sus usuarios en un RDD que tiene registros con el siguiente formato: (chat_id, user_id, nickname, text). Queremos saber cuál es el usuario (user_id) que mas preguntas hace en sus chats, contabilizamos una pregunta por cada caracter “?” que aparezca en el campo text. Programar en Spark un programa que identifique al usuario preguntón.

In [41]:
chats = [
    (1, 1, 'damu', 'Qué es esto?'),
    (2, 2, 'martin', 'Un chat!???'),
    (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
    (4, 2, 'martin', 'Sí! Cómo sabias????'),
    (5, 1, 'damu', 'Adivine'),
    (6, 3, 'luis', 'Hola!')
]

In [42]:
data = sc.parallelize(chats)

In [43]:
data.collect()

[(1, 1, 'damu', 'Qué es esto?'),
 (2, 2, 'martin', 'Un chat!???'),
 (3, 1, 'damu', 'Ahhh! Y de donde salio? Whatsapp?'),
 (4, 2, 'martin', 'Sí! Cómo sabias????'),
 (5, 1, 'damu', 'Adivine'),
 (6, 3, 'luis', 'Hola!')]

In [45]:
# Por cada registro guardo (id del usuario, preguntas por registro)
data.map(lambda x: (x[1], x[3].count('?'))).collect()

[(1, 1), (2, 3), (1, 2), (2, 4), (1, 0), (3, 0)]

In [46]:
# Cuenta cuantas preguntas hizo cada usuario
data.map(lambda x: (x[1], x[3].count('?'))).reduceByKey(lambda x, y: x + y).collect()

[(1, 3), (2, 7), (3, 0)]

In [49]:
# El mayor
data.map(lambda x: (x[1], x[3].count('?'))).reduceByKey(lambda x, y: x + y)\
    .reduce(lambda x, y: x if x[1] > y[1] else y)

(2, 7)